#  SPARQL from Python

`SPARQLWrapper` is a simple Python wrapper around a SPARQL service for remote query execution. Not only does it enable us to write more complex queries to extract information from RDF than those exposed through a library like `rdflib`, it can also convert query results into other formats like JSON and CSV!

## First, what is SPARQL?

The RDF specification defines the syntax and semantics of the SPARQL query language. SPARQL can be used to express queries across diverse data sources, whether the data is stored natively as RDF or viewed as RDF via middleware. SPARQL contains capabilities for querying required and optional graph patterns along with their conjunctions and disjunctions. SPARQL also supports extensible value testing and constraining queries by source RDF graph. The results of SPARQL queries can be results sets or RDF graphs.

## `SPARQLWrapper`

The Python library `SPARQLWrapper` will enable us to use the SPARQL query language without the need for any additional tools (like Apache Jena).

    pip install sparqlwrapper



In [7]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Capsaicin> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results['head']:
    print(result)

link
vars


In [ ]:
for result in results["results"]["bindings"]:
    print(result["label"]["value"])